# MLEM2 Algorithm

In [1]:
import pandas as pd
from pandas.api.types import is_numeric_dtype

In [2]:
df = pd.read_csv('../data/trip_data.csv')

In [3]:
df = df.drop(['Case'],axis=1)

In [4]:
df

,Wind,Humidity,Temperature,Trip
0,4,low,medium,yes
1,8,low,low,yes
2,4,medium,medium,yes
3,8,medium,high,maybe
4,12,low,medium,maybe
5,16,high,low,no
6,30,high,high,no
7,12,high,high,no


#### -------------------Finding numeric column---------------------

In [5]:
numeric_col = df.dtypes[df.dtypes != "object"].index[0]

In [6]:
numeric_col

'Wind'

In [7]:
sort_col = df[numeric_col].sort_values()

In [8]:
df['sort_col'] = sort_col.values

In [9]:
df

,Wind,Humidity,Temperature,Trip,sort_col
0,4,low,medium,yes,4
1,8,low,low,yes,4
2,4,medium,medium,yes,8
3,8,medium,high,maybe,8
4,12,low,medium,maybe,12
5,16,high,low,no,12
6,30,high,high,no,16
7,12,high,high,no,30


In [10]:
point_list = df['sort_col'].unique()

In [11]:
point_list

array([ 4,  8, 12, 16, 30])

In [12]:
avg_list = []
for i in range(len(point_list)-1):
    avg = (point_list[i] + point_list[i+1])/2
    avg_list.append(int(avg))

In [13]:
avg_list

[6, 10, 14, 23]

In [14]:
attributes = list(df)
attributes

['Wind', 'Humidity', 'Temperature', 'Trip', 'sort_col']

In [15]:
case_list = []
for i in avg_list:
    case = str(numeric_col) + "," + str(point_list[0]) + ".." + str(i)
    case2 = str(numeric_col) + "," + str(i) + ".." + str(point_list[len(point_list)-1])
    case_list.append(case)
    case_list.append(case2)

In [16]:
case_list

['Wind,4..6',
 'Wind,6..30',
 'Wind,4..10',
 'Wind,10..30',
 'Wind,4..14',
 'Wind,14..30',
 'Wind,4..23',
 'Wind,23..30']

In [17]:
for item in attributes[:-2]:
    print(item)
    #check for non numeric columns
    if not is_numeric_dtype(df[item]):
        temp = df[item].unique()
        for i in temp:
            case = item + "," + i
            case_list.append(case)
        

Wind
Humidity
Temperature


In [18]:
case_list

['Wind,4..6',
 'Wind,6..30',
 'Wind,4..10',
 'Wind,10..30',
 'Wind,4..14',
 'Wind,14..30',
 'Wind,4..23',
 'Wind,23..30',
 'Humidity,low',
 'Humidity,medium',
 'Humidity,high',
 'Temperature,medium',
 'Temperature,low',
 'Temperature,high']

#### ---------------Building (a,v) pairs---------------------

In [19]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if "." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if row[a] >= int(start) and row[a] <= int(end):
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []
#         print("\n")
        

[1, 3]
[2, 4, 5, 6, 7, 8]
[1, 2, 3, 4]
[5, 6, 7, 8]
[1, 2, 3, 4, 5, 8]
[6, 7]
[1, 2, 3, 4, 5, 6, 8]
[7]


In [20]:
att_val_list

[[1, 3],
 [2, 4, 5, 6, 7, 8],
 [1, 2, 3, 4],
 [5, 6, 7, 8],
 [1, 2, 3, 4, 5, 8],
 [6, 7],
 [1, 2, 3, 4, 5, 6, 8],
 [7]]